In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from typing import Any, Iterable

import block_formats.experiments as E

def mean_and_stderr(samples: list[float]) -> tuple[float, float]:
    t = torch.tensor(samples)
    return t.mean().item(), t.std(correction=0).div(len(t)**.5).item()

/home/ubuntu/SquashedLlama/notebooks/20250127-FormatsCascade/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
def get_result(run: dict[str, Any]) -> dict[str, Any]:
    return dict(
        id=run.id,
        model=run.config.model,
        fmt=run.config.test.get("fmt_str", "bfloat16"),
        duration=run.meta.duration,
        **dict(zip(["xent", "xent_stderr"], mean_and_stderr(run.summary.cross_entropy))),
        **dict(zip(["kl_div", "kl_div_stderr"], mean_and_stderr(run.summary.kl_div))),
    )

df = pd.DataFrame.from_records(map(get_result, E.runs("20250422-update-models")))

## Baseline performance

In [28]:
display(df[df.fmt == "bfloat16"].sort_values("xent")[["model", "xent", "xent_stderr", "duration"]])

,model,xent,xent_stderr,duration
25,google/gemma-3-12b-pt,1.698757,0.030636,66.280640
10,meta-llama/Llama-3.1-8B,1.757471,0.029548,29.120567
50,microsoft/phi-4,1.795534,0.027184,53.006662
45,Qwen/Qwen2.5-7B,1.851347,0.027800,28.172443
20,google/gemma-3-4b-pt,1.923519,0.030846,23.538167
5,meta-llama/Llama-3.2-3B,1.973079,0.029390,13.393867
40,Qwen/Qwen2.5-3B,2.008395,0.028012,13.909998
35,Qwen/Qwen2.5-1.5B,2.146768,0.028958,7.658893
0,meta-llama/Llama-3.2-1B,2.195859,0.030371,5.343262
15,google/gemma-3-1b-pt,2.275802,0.032057,8.542517


## Quantisation performance

In [39]:
(df.pivot(index="model", columns="fmt", values="kl_div")
 [["bfloat16", "4b-int+Zoptimal{1,*:BFLOAT16:rms}", "E2M1{1,64:BFLOAT16:absmax}", "3b-int+Zoptimal{1,*:BFLOAT16:rms}", "E0M2{1,64:BFLOAT16:signmax}"]]
 .style.background_gradient(axis=None, vmin=0, vmax=1)
)

fmt,bfloat16,"4b-int+Zoptimal{1,*:BFLOAT16:rms}","E2M1{1,64:BFLOAT16:absmax}","3b-int+Zoptimal{1,*:BFLOAT16:rms}","E0M2{1,64:BFLOAT16:signmax}"
model,,,,,
Qwen/Qwen2.5-0.5B,0.000000,0.106019,0.218543,0.630242,1.167914
Qwen/Qwen2.5-1.5B,0.000000,0.086550,0.158438,0.353764,0.799734
Qwen/Qwen2.5-3B,0.000000,0.065924,0.139068,0.315227,3.557490
Qwen/Qwen2.5-7B,0.000000,0.044011,0.112279,0.185150,0.503392
google/gemma-3-12b-pt,0.000000,0.046728,0.090392,0.179889,0.470199
google/gemma-3-1b-pt,0.000000,0.075709,0.143394,0.466562,0.862925
google/gemma-3-4b-pt,0.000000,0.046307,0.092750,0.213687,0.440094
meta-llama/Llama-3.1-8B,0.000000,0.049122,0.088664,2.837985,0.445041
meta-llama/Llama-3.2-1B,0.000000,0.087402,0.135905,0.405110,0.986572
